### Import of necessary libraries

In [1]:
from interface import Interface
from shapely import Polygon
import folium
import geopandas

### Creation of a polygon as an example of AOI

In [2]:
# simplified example of a call to the interface

lat_point_list = [50.854457, 50.924457, 52.518172, 50.072651, 48.853033]
lon_point_list = [4.377184, 4.377184, 13.407759, 14.435935, 2.349553]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))

polygon_geom = geopandas.GeoSeries(polygon_geom)

m = folium.Map(location=[50.854457, 4.377184], zoom_start=5)
folium.GeoJson(polygon_geom.__geo_interface__).add_to(m)

x = geopandas.GeoSeries(polygon_geom)
              
rectangle = geopandas.GeoDataFrame(geometry=x.envelope)

# Define the number of rows and columns for the grid
num_rows = 5
num_cols = 5

# Calculate the width and height of each sub-area
width = (rectangle.bounds['maxx'] - rectangle.bounds['minx']) / num_cols
height = (rectangle.bounds['maxy'] - rectangle.bounds['miny']) / num_rows

# Create an empty list to store the sub-areas
sub_areas = []

# Iterate over the rows and columns to create the grid of sub-areas
for row in range(num_rows):
    for col in range(num_cols):
        minx = rectangle.bounds['minx'] + col * width
        maxx = minx + width
        miny = rectangle.bounds['miny'] + row * height
        maxy = miny + height
        sub_area_coords = [(minx, miny), (maxx, miny), (maxx, maxy), (minx, maxy)]
        sub_areas.append(Polygon(sub_area_coords))

# Create a GeoDataFrame with the sub-areas
sub_areas_gdf = geopandas.GeoDataFrame(geometry=sub_areas)

# Perform the spatial intersection to obtain the divided sub-areas
divided_areas = geopandas.overlay(sub_areas_gdf, rectangle, how='intersection')

# print(divided_areas)

for index, row in divided_areas.iterrows():
    geojson = row.geometry.__geo_interface__
    folium.GeoJson(geojson).add_to(m)
    
centroids = set()

for index, row in divided_areas.iterrows():    
    if(polygon_geom.intersects(row.geometry).bool()==True):
        centroids.add(row.geometry.centroid)

for centroid in centroids:
    print(centroid)
    folium.GeoJson(centroid).add_to(m)

# folium.GeoJson(divided_areas.to_json).add_to(m)
   
m

POINT (8.392744 49.219546900000005)
POINT (5.975467600000001 51.41863030000001)
POINT (13.227296800000001 52.151658100000006)
POINT (13.227296800000001 49.95257470000001)
POINT (13.227296800000001 50.685602500000016)
POINT (5.975467600000001 49.219546900000005)
POINT (3.5581912 49.95257470000001)
POINT (8.392744 50.685602500000016)
POINT (8.392744 52.151658100000006)
POINT (10.8100204 49.219546900000005)
POINT (5.975467600000001 50.68560250000001)
POINT (10.8100204 50.685602500000016)
POINT (13.227296800000001 51.418630300000004)
POINT (8.392744 49.95257470000001)
POINT (3.5581912 50.6856025)
POINT (3.5581912 49.2195469)
POINT (5.975467600000001 49.9525747)
POINT (10.8100204 52.151658100000006)
POINT (8.392744 51.418630300000004)
POINT (10.8100204 51.418630300000004)
POINT (10.8100204 49.95257470000001)


### The class interface is created and the method get_weather_forecast is called by group 6

In [3]:
interface = Interface()
interface.get_weather_forecast(polygon_geom)

{'u1', 'u0', 'u2', 'u3'}


,time,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,...,soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_1cm,soil_moisture_1_3cm,soil_moisture_3_9cm,soil_moisture_9_27cm,soil_moisture_27_81cm,is_day,AOI_ID,EventID
0,2023-05-17T00:00,8.8,74,4.4,3.4,0,0.0,0.0,0.0,0.0,...,11.0,11.3,0.270,0.272,0.277,0.293,0.326,0,1,1
1,2023-05-17T01:00,8.8,74,4.5,3.5,0,0.0,0.0,0.0,0.0,...,10.9,11.3,0.270,0.272,0.277,0.293,0.326,0,1,1
2,2023-05-17T02:00,8.4,73,3.8,3.3,0,0.0,0.0,0.0,0.0,...,10.8,11.3,0.270,0.272,0.277,0.292,0.326,0,1,1
3,2023-05-17T03:00,8.3,70,3.2,3.1,0,0.0,0.0,0.0,0.0,...,10.6,11.3,0.279,0.281,0.285,0.297,0.327,0,1,1
4,2023-05-17T04:00,8.0,68,2.6,3.0,0,0.0,0.0,0.0,0.0,...,10.5,11.3,0.279,0.281,0.285,0.297,0.327,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2023-05-23T19:00,11.6,92,10.3,7.6,5,0.0,0.0,0.0,0.0,...,12.4,12.4,0.239,0.240,0.246,0.269,0.312,1,1,1
164,2023-05-23T20:00,11.4,92,10.2,7.1,4,0.0,0.0,0.0,0.0,...,12.4,12.4,0.239,0.240,0.246,0.269,0.312,0,1,1
165,2023-05-23T21:00,11.2,92,10.0,6.8,3,0.0,0.0,0.0,0.0,...,12.4,12.4,0.239,0.240,0.246,0.269,0.312,0,1,1
166,2023-05-23T22:00,11.2,91,9.8,7.1,3,0.0,0.0,0.0,0.0,...,12.4,12.4,0.239,0.240,0.246,0.269,0.312,0,1,1
